# Task 1

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import json
import numpy as np
from scipy.stats import pearsonr

In [ ]:
#Importing Feature Geometry Shapefiles
catchments_primary = gpd.read_file("Catchments/catchments_primary.shp")
catchments_secondary = gpd.read_file("Catchments/catchments_secondary.shp")
catchments_future = gpd.read_file("Catchments/catchments_future.shp")
sa2_boundaries = gpd.read_file("SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
unemployment = gpd.read_file("Data/SA2_unemployment.shp")

In [ ]:
#Importing csv, txt files
businesses=pd.read_csv("Businesses.csv")
income=pd.read_csv("Income.csv")
polling=pd.read_csv("PollingPlaces2019.csv")
population=pd.read_csv("Population.csv")
toilets = pd.read_csv('Australia Public Toilet Map.csv')
stops=pd.read_csv("Stops.txt", sep=",")

In [ ]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [ ]:
db, conn = pgconnect(credentials)

### Traffic

In [ ]:
with open("live-traffic-cameras.json") as traffic:
    data = json.load(traffic)


In [ ]:
traffic = pd.DataFrame(columns=["geo_point_2d", "region", "title", "view", "direction", "href", "photo"])

for i in range(0, len(data)):
    currentItem = data[i]
    traffic.loc[i] = [data[i]["geo_point_2d"], data[i]["region"], data[i]["title"], data[i]["view"], data[i]["direction"], data[i]["href"], data[i]["photo"]]


In [ ]:
traffic[['lng', 'lat']] = traffic["geo_point_2d"].apply(pd.Series)
traffic['geom'] = gpd.points_from_xy(traffic.lng, traffic.lat)
traffic = traffic.drop(columns=['geo_point_2d', 'lng', 'lat', 'photo', 'href'])
traffic['id'] = pd.Series(range(1, len(traffic)+1))
traffic['geom'] = traffic['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))
traffic

In [ ]:
sql = """
DROP TABLE IF EXISTS traffic;
CREATE TABLE traffic (
    id INTEGER PRIMARY KEY,
    region VARCHAR(100), 
    title VARCHAR(100), 
    view TEXT, 
    direction VARCHAR(3),
    geom GEOMETRY(POINT,4326)
);"""

query(conn, sql)

In [ ]:
sql = """
CREATE INDEX traffic_geom_idx ON traffic USING GIST(geom);
"""
query(conn, sql)

In [ ]:
traffic.to_sql('traffic', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4326)})
query(conn, "select * from traffic")

### Catchments

In [ ]:
catchments_primary=catchments_primary.drop(catchments_primary.loc[:,'ADD_DATE':'PRIORITY'].columns, axis=1)
catchments_primary=catchments_primary.rename(columns={'USE_ID':'use_id', 'CATCH_TYPE':'catch_type', 'USE_DESC':'use_desc'})

In [ ]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

catchments_primary['geom'] = catchments_primary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4326))
catchments_primary=catchments_primary.drop(columns="geometry")

In [ ]:
sql = """
DROP TABLE IF EXISTS catchments_primary;
CREATE TABLE catchments_primary (
    USE_ID INTEGER PRIMARY KEY,
    CATCH_TYPE VARCHAR(20),
    USE_DESC VARCHAR(50), 
    geom GEOMETRY(MULTIPOLYGON,4326)

);"""

query(conn, sql)

In [ ]:
sql = """
CREATE INDEX catchments_primary_geom_idx ON catchments_primary USING GIST(geom);
"""
query(conn, sql)

In [ ]:
catchments_primary.to_sql('catchments_primary', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)})
query(conn, "select * from catchments_primary")

In [ ]:
catchments_secondary=catchments_secondary.drop(catchments_secondary.loc[:,'ADD_DATE':'PRIORITY'].columns, axis=1)
catchments_secondary=catchments_secondary.rename(columns={'USE_ID':'use_id', 'CATCH_TYPE':'catch_type', 'USE_DESC':'use_desc'})

In [ ]:
catchments_secondary['geom'] = catchments_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4326))
catchments_secondary=catchments_secondary.drop(columns="geometry")

In [ ]:
sql = """
DROP TABLE IF EXISTS catchments_secondary;
CREATE TABLE catchments_secondary (
    USE_ID INTEGER PRIMARY KEY,
    CATCH_TYPE VARCHAR(20),
    USE_DESC VARCHAR(50),
    geom GEOMETRY(MULTIPOLYGON,4326)
);"""


query(conn, sql)

In [ ]:
sql = """
CREATE INDEX catchments_secondary_geom_idx ON catchments_secondary USING GIST(geom);
"""
query(conn, sql)

In [ ]:
catchments_secondary.to_sql('catchments_secondary', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)})
query(conn, "select * from catchments_secondary")

In [ ]:
catchments_future=catchments_future.drop(catchments_future.loc[:,'ADD_DATE':'YEAR12'].columns, axis=1)
catchments_future=catchments_future.rename(columns={'USE_ID':'use_id', 'CATCH_TYPE':'catch_type', 'USE_DESC':'use_desc'})

In [ ]:
catchments_future['geom'] = catchments_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4326))
catchments_future=catchments_future.drop(columns="geometry")

In [ ]:
sql = """
DROP TABLE IF EXISTS catchments_future;
CREATE TABLE catchments_future (
    USE_ID INTEGER PRIMARY KEY,
    CATCH_TYPE VARCHAR(20),
    USE_DESC VARCHAR(50),
    geom GEOMETRY(MULTIPOLYGON,4326)
);"""

query(conn, sql)

In [ ]:
sql = """
CREATE INDEX catchments_future_geom_idx ON catchments_future USING GIST(geom);
"""
query(conn, sql)

In [ ]:
catchments_future.to_sql('catchments_future', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)})
query(conn, "select * from catchments_future")

### SA2 Boundaries

In [ ]:
sa2_boundaries = sa2_boundaries.drop(['CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21', 'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'GCC_CODE21', 'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'LOCI_URI21'], axis = 1)
sa2_boundaries = sa2_boundaries[sa2_boundaries['GCC_NAME21'] == 'Greater Sydney']
sa2_boundaries['SA2_CODE21']=sa2_boundaries['SA2_CODE21'].astype(np.int64)
sa2_boundaries = sa2_boundaries.drop(['GCC_NAME21'], axis = 1)

In [ ]:
sa2_boundaries['geom'] = sa2_boundaries['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4326))

In [ ]:
sa2_boundaries = sa2_boundaries.drop(['geometry'], axis = 1)
sa2_boundaries = sa2_boundaries.rename(columns={'SA2_CODE21': 'sa2_code21', 'SA2_NAME21': 'sa2_name21', 'AREASQKM21': 'areasqkm21'})

In [ ]:
sql = """
DROP TABLE IF EXISTS sa2_boundaries;
CREATE TABLE sa2_boundaries (
    sa2_code21 INTEGER PRIMARY KEY, 
    sa2_name21 VARCHAR(100), 
    areasqkm21 FLOAT,
    geom GEOMETRY(MULTIPOLYGON,4326)
);"""

query(conn, sql)

In [ ]:
sql = """
CREATE INDEX sa2_boundaries_geom_idx ON sa2_boundaries USING GIST(geom);
"""
query(conn, sql)

In [ ]:
sa2_boundaries.to_sql('sa2_boundaries', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)})
query(conn, "select * from sa2_boundaries")

### Income

In [ ]:
income=income.replace({'np': None})
income=income.dropna() #rows containing np did not contain integer values, hence removal
income=income[income['sa2_code'].isin(sa2_boundaries['sa2_code21'])]

In [ ]:
sql = """
DROP TABLE IF EXISTS income;
CREATE TABLE income(
    sa2_code INTEGER PRIMARY KEY,
    sa2_name VARCHAR(100), 
    earners INTEGER, 
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER,
    
    CONSTRAINT fkey_sa2inc FOREIGN KEY(sa2_code)
        REFERENCES sa2_boundaries(sa2_code21)
);"""


query(conn, sql)

In [ ]:
income.to_sql('income', conn, if_exists='append', index=False)
query(conn, "select * from income")

### Businesses

In [ ]:
businesses=businesses.drop(businesses.columns[[0]], axis=1) #dropped industry code to remove repeated information
#created a primary key from industry name and sa2 code columns, since no single column was a unique identifier for records
businesses=businesses[businesses['sa2_code'].isin(sa2_boundaries['sa2_code21'])]

In [ ]:
sql = """
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses(
    industry_name VARCHAR(100),
    sa2_code INTEGER, 
    sa2_name VARCHAR(100),
    "0_to_50k_businesses" INTEGER,
    "50k_to_200k_businesses" INTEGER,
    "200k_to_2m_businesses" INTEGER,
    "2m_to_5m_businesses" INTEGER,
    "5m_to_10m_businesses" INTEGER,
    "10m_or_more_businesses" INTEGER,
    total_businesses INTEGER,
    PRIMARY KEY (industry_name, sa2_code),
    
    CONSTRAINT fkey_sa2bus FOREIGN KEY(sa2_code)
        REFERENCES sa2_boundaries(sa2_code21)
);"""

query(conn, sql)

In [ ]:
businesses.to_sql('businesses', conn, if_exists='append', index=False)
query(conn, "select * from businesses")

### Population

In [ ]:
sql = """
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code INTEGER PRIMARY KEY, 
    sa2_name VARCHAR(100), 
    "0-4_people" INTEGER, 
    "5-9_people" INTEGER, 
    "10-14_people" INTEGER,
    "15-19_people" INTEGER, 
    "20-24_people" INTEGER, 
    "25-29_people" INTEGER, 
    "30-34_people" INTEGER, 
    "35-39_people" INTEGER, 
    "40-44_people" INTEGER, 
    "45-49_people" INTEGER, 
    "50-54_people" INTEGER, 
    "55-59_people" INTEGER, 
    "60-64_people" INTEGER, 
    "65-69_people" INTEGER, 
    "70-74_people" INTEGER, 
    "75-79_people" INTEGER, 
    "80-84_people" INTEGER, 
    "85-and-over_people" INTEGER, 
    total_people INTEGER,
    
    CONSTRAINT fkey_sa2pop FOREIGN KEY(sa2_code)
        REFERENCES sa2_boundaries(sa2_code21)
);"""

query(conn, sql)

In [ ]:
population.to_sql('population', conn, if_exists='append', index=False)
query(conn, "select * from population")

### Toilets...

In [ ]:
toilets = toilets.drop(['URL', 'FacilityType', 'AddressNote', 'Parking', 'ParkingNote', 'KeyRequired', 'MLAK24', 'MLAKAfterHours', 'PaymentRequired', 'AccessNote', 'AdultChange', 'ChangingPlaces', 'BYOSling', 'ACShower', 'ACMLAK', 'AdultChangeNote', 'BabyChange', 'BabyCareRoom', 'BabyChangeNote', 'DumpPoint', 'DPWashout', 'DPAfterHours', 'DumpPointNote', 'OpeningHours', 'OpeningHoursNote', 'Male', 'Female', 'Unisex', 'AllGender', 'Ambulant', 'Accessible', 'LHTransfer', 'RHTransfer', 'ToiletNote', 'SharpsDisposal', 'DrinkingWater', 'SanitaryDisposal', 'MensPadDisposal', 'Shower', 'ParkingAccessible'  ], axis=1)

In [ ]:
#Turning toilet latitude and longitude to point data
toilets['geom'] = gpd.points_from_xy(toilets.Longitude, toilets.Latitude)
toilets = toilets.drop(['Latitude', 'Longitude'], axis = 1)
toilets['geom'] = toilets['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))

#Filtering out toilets in NSW
toilets = toilets[toilets['State'] == 'NSW']
toilets = toilets.drop(['State'], axis = 1)

In [ ]:
toilets = toilets.rename(columns={'FacilityID': 'facilityid','Name': 'name','Address1': 'address1','Town': 'town'})

In [ ]:
toilets

In [ ]:
sql = """
DROP TABLE IF EXISTS toilets;
CREATE TABLE toilets (
    facilityid INTEGER PRIMARY KEY, 
    name VARCHAR(100), 
    address1 VARCHAR(100),
    town VARCHAR(50),
    geom GEOMETRY(POINT,4326)
);"""

query(conn, sql)

In [ ]:
sql = """
CREATE INDEX toilets_geom_idx ON toilets USING GIST(geom);
"""
query(conn, sql)

In [ ]:
toilets.to_sql('toilets', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4326)})
query(conn, "select * from toilets")

### Polling

In [ ]:
polling = polling.drop(['the_geom','FID', 'state', 'division_id', 'polling_place_type_id', 'premises_address_1', 'premises_address_2', 'premises_address_3', 'premises_state_abbreviation', 'premises_post_code'], axis = 1)

In [ ]:
polling['geom'] = gpd.points_from_xy(polling.longitude, polling.latitude)
polling = polling.drop(['longitude', 'latitude'], axis = 1)
polling['geom'] = polling['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))

In [14]:
sql = """
DROP TABLE IF EXISTS polling;
CREATE TABLE polling (
    polling_place_id INTEGER PRIMARY KEY, 
    division_name VARCHAR(20), 
    polling_place_name VARCHAR(100),
    premises_name VARCHAR(100),
    premises_suburb VARCHAR(100),
    geom GEOMETRY(POINT,4326)
);"""


query(conn, sql)

NameError: name 'query' is not defined

In [ ]:
sql = """
CREATE INDEX polling_geom_idx ON polling USING GIST(geom);
"""
query(conn, sql)

In [ ]:
polling.to_sql('polling', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4326)})
query(conn, "select * from polling")

### Stops

In [ ]:
stops = stops.drop(['stop_code', 'location_type', 'parent_station', 'wheelchair_boarding', 'platform_code'], axis = 1)

In [ ]:
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
stops = stops.drop(['stop_lon', 'stop_lat'], axis = 1)
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))


In [ ]:
sql = """
DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stop_id TEXT PRIMARY KEY, 
    stop_name VARCHAR(100), 
    geom GEOMETRY(POINT,4326)
);"""

query(conn, sql)

In [ ]:
sql = """
CREATE INDEX stops_geom_idx ON stops USING GIST(geom);
"""
query(conn, sql)

In [ ]:
stops.to_sql('stops', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4326)})
query(conn, "select * from stops")

### Unemployment

In [ ]:
#Dropping SA2_MAIN16 & STATE_CODE columns
unemployment=unemployment.drop(['SA2_MAIN16', 'STATE_CODE'], axis=1)

In [ ]:
#Renaming columns
unemployment=unemployment.rename(columns={'SA2_MAIN':'sa2_code', 'SA2_NAME':'sa2_name', 'STATE_NAME':'state_name', 'AREA_SQKM':'area_sqkm', 'PER_UNEMPL':'per_unempl'})
#Converting SA2 code data type to integer
unemployment['sa2_code']=unemployment['sa2_code'].astype(np.int64)
#Retaining only the rows for which SA2 code exists in the boundaries dataset
unemployment=unemployment[unemployment['sa2_code'].isin(sa2_boundaries['sa2_code21'])]
unemployment

In [ ]:
#Converting polygons to multipolygons
unemployment['geom'] = unemployment['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4326))
unemployment = unemployment.drop(['geometry'], axis = 1)

In [ ]:
sql = """

DROP TABLE IF EXISTS unemployment;
CREATE TABLE unemployment (
    sa2_code INTEGER PRIMARY KEY,
    sa2_name VARCHAR(100),
    state_name VARCHAR(100),
    area_sqkm FLOAT,
    per_unempl FLOAT,
    geom GEOMETRY(MULTIPOLYGON,4326),
    
    CONSTRAINT fkey_sa2une FOREIGN KEY(sa2_code)
        REFERENCES sa2_boundaries(sa2_code21)
);"""

query(conn, sql)

In [ ]:
sql = """
CREATE INDEX unemployment_geom_idx ON unemployment USING GIST(geom);
"""
query(conn, sql)

In [ ]:
unemployment.to_sql('unemployment', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)})
query(conn, "select * from unemployment")

# Tasks 2 & 3

### SQL Results Calculation

In [ ]:
unextended_data = pd.read_sql_query("""

WITH filtered_stops AS (
	SELECT sa2_boundaries.sa2_code21 AS sa2_code, COUNT(*) AS stops_total FROM sa2_boundaries 
	JOIN stops
	ON ST_Contains(sa2_boundaries.geom, stops.geom)
	GROUP BY sa2_code21
),

filtered_businesses AS (
	SELECT a.sa2_code, 
		a.sa2_name, 
		a.total_businesses AS retail_total_businesses, 
		b.total_businesses as health_total_businesses, total_people, 
		(cast(b.total_businesses as float) / total_people) * 1000 as health_businesses_per_1000, 
		(cast(a.total_businesses as float) / total_people) * 1000 as retail_businesses_per_1000 
	FROM businesses a, businesses b, population
	WHERE a.sa2_code = b.sa2_code and b.sa2_code = population.sa2_code
	AND a.industry_name = 'Health Care and Social Assistance' AND b.industry_name = 'Retail Trade'
	and total_people >= 100
), 

filtered_pol AS (
	SELECT sa2_code21, count(*) AS polling_total FROM sa2_boundaries 
	JOIN polling
	ON ST_Contains(sa2_boundaries.geom, polling.geom)
	GROUP BY sa2_code21
), 
	
filtered_primary AS ( 
	select sa2_code21, 
		COUNT(catchments_primary.use_id) AS primary_catchments
	FROM sa2_boundaries 
	join catchments_primary
	on ST_intersects(sa2_boundaries.geom, catchments_primary.geom)
	group by sa2_code21
), 

primary_reduced_population as (
select sa2_code,
	(cast(primary_catchments as float)/("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") ) * 1000 as primary_per_1000
	from filtered_primary
	join population 
	on filtered_primary.sa2_code21 = population.sa2_code
	where total_people >= 100	
),

filtered_secondary AS (
	SELECT sa2_code21, COUNT(*) AS secondary_catchments FROM sa2_boundaries 
	JOIN catchments_secondary
	ON ST_Intersects(sa2_boundaries.geom, catchments_secondary.geom)
	GROUP BY sa2_code21
), 

secondary_reduced_population as (
select sa2_code,
	(cast(secondary_catchments as float)/("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") ) * 1000 as secondary_per_1000
	from filtered_secondary
	join population 
	on filtered_secondary.sa2_code21 = population.sa2_code
	where total_people >= 100	
),

filtered_future AS (
	SELECT sa2_code21, COUNT(*) AS future_catchments FROM sa2_boundaries 
	JOIN catchments_future
	ON ST_intersects(sa2_boundaries.geom, catchments_future.geom)
	GROUP BY sa2_code21
),

future_reduced_population as (
select sa2_code,
	(cast(future_catchments as float)/("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") ) * 1000 as future_per_1000
	from filtered_future
	join population 
	on filtered_future.sa2_code21 = population.sa2_code
	where total_people >= 100	
),

filtered_toilets AS (
	SELECT sa2_code21, COUNT(*) AS toilet_total FROM sa2_boundaries 
	JOIN toilets
	ON ST_contains(sa2_boundaries.geom, toilets.geom)
	GROUP BY sa2_code21
),

filtered_unemployment AS (
	SELECT population.sa2_code, 
	population.sa2_name, 
	"per_unempl", 
	(100 - per_unempl) as employment_percentage,
	(total_people - "0-4_people" - "5-9_people" - "10-14_people") AS adult_population, 
	ROUND(((total_people - "0-4_people" - "5-9_people" - "10-14_people") - ((per_unempl / 100) * (total_people - "0-4_people" - "5-9_people" - "10-14_people")))) AS total_employed 
	FROM  population
	join unemployment
	ON population.sa2_code = unemployment.sa2_code
	where total_people >= 100
),

filtered_traffic as (
	SELECT sa2_code21, sa2_name21, COUNT(*) AS traffic_camera_count FROM sa2_boundaries 
	JOIN traffic
	ON ST_intersects(sa2_boundaries.geom, traffic.geom)
	GROUP BY sa2_code21, sa2_name21
),


zscore_stops AS (
SELECT sa2_code, 
	stops_total,
	(stops_total - (select avg(stops_total) from filtered_stops)) / (select stddev(stops_total) from filtered_stops) as z_score_stops
FROM filtered_stops
), 

zscore_businesses AS (
SELECT sa2_code, 
	retail_total_businesses, 
	(retail_businesses_per_1000 - (select avg(retail_businesses_per_1000) from filtered_businesses)) / (select stddev(retail_businesses_per_1000) from filtered_businesses) as z_score_retail, 
	health_total_businesses,
	(health_businesses_per_1000 - (select avg(health_businesses_per_1000) from filtered_businesses)) / (select stddev(health_businesses_per_1000) from filtered_businesses) as z_score_health 
FROM filtered_businesses
),

zscore_pol AS (
SELECT sa2_code21, 
	polling_total,
	(polling_total - (select avg(polling_total) from filtered_pol)) / (select stddev(polling_total) from filtered_pol) as z_score_pol
FROM filtered_pol
), 

zscore_primary AS (
SELECT sa2_code, 
	primary_per_1000,
	(primary_per_1000 - (select avg(primary_per_1000) from primary_reduced_population)) / (select stddev(primary_per_1000) from primary_reduced_population) as z_score_primary
FROM primary_reduced_population
),

zscore_secondary AS (
SELECT sa2_code, 
	secondary_per_1000,
	(secondary_per_1000 - (select avg(secondary_per_1000) from secondary_reduced_population)) / (select stddev(secondary_per_1000) from secondary_reduced_population) as z_score_secondary
FROM secondary_reduced_population
),

zscore_future AS (
SELECT sa2_code, 
	future_per_1000,
	(future_per_1000 - (select avg(future_per_1000) from future_reduced_population)) / (select stddev(future_per_1000) from future_reduced_population) as z_score_future
FROM future_reduced_population
),

zscore_toilets AS (
SELECT sa2_code21, 
	toilet_total,
	(toilet_total - (select avg(toilet_total) from filtered_toilets)) / (select stddev(toilet_total) from filtered_toilets) as z_score_toilets
FROM filtered_toilets
), 

zscore_employment AS (
SELECT sa2_code, 
	employment_percentage,
	(employment_percentage - (select avg(employment_percentage) from filtered_unemployment)) / (select stddev(employment_percentage) from filtered_unemployment) as z_score_employment
FROM filtered_unemployment
),

zscore_traffic AS (
SELECT sa2_code21, 
	traffic_camera_count,
	(traffic_camera_count - (select avg(traffic_camera_count) from filtered_traffic)) / (select stddev(traffic_camera_count) from filtered_traffic) as z_score_traffic
FROM filtered_traffic
), 


total_zscore as ( 
	SELECT 
		sa2_boundaries.sa2_code21,
        sa2_boundaries.geom,
        sa2_boundaries.sa2_name21,
		stops_total, 
		z_score_stops,
		retail_total_businesses,
		z_score_retail,
		health_total_businesses, 
		z_score_health,
		polling_total,
		z_score_pol,
		primary_per_1000, 
		z_score_primary,
		secondary_per_1000, 
		z_score_secondary,
		future_per_1000, 
		z_score_future,
		(COALESCE(z_score_stops, 0) + COALESCE(z_score_retail, 0) + COALESCE(z_score_health, 0) + COALESCE(z_score_pol, 0) + COALESCE(z_score_primary, 0) + COALESCE(z_score_secondary, 0) + COALESCE(z_score_future, 0)) AS x_val
	FROM sa2_boundaries
	LEFT JOIN zscore_stops
	ON sa2_boundaries.sa2_code21 = zscore_stops.sa2_code
	LEFT JOIN zscore_businesses 
	ON sa2_boundaries.sa2_code21 = zscore_businesses.sa2_code
	LEFT JOIN zscore_pol 
	ON sa2_boundaries.sa2_code21 = zscore_pol.sa2_code21
	LEFT JOIN zscore_primary 
	ON sa2_boundaries.sa2_code21 = zscore_primary.sa2_code
	LEFT JOIN zscore_secondary
	ON sa2_boundaries.sa2_code21 = zscore_secondary.sa2_code
	LEFT JOIN zscore_future
	ON sa2_boundaries.sa2_code21 = zscore_future.sa2_code
    LEFT JOIN income
    ON sa2_boundaries.sa2_code21 = income.sa2_code
)

select *, (1 / (1 + exp(-"x_val"))) as sigmoid_val from total_zscore



""",conn)

In [ ]:
sql = """

DROP TABLE IF EXISTS unextended_data;
CREATE TABLE unextended_data (
    sa2_code21 INTEGER PRIMARY KEY,
    sa2_name21 VARCHAR(100),
    stops_total INTEGER,
    z_score_stops FLOAT,
    retail_total_businesses INTEGER,
    z_score_retail FLOAT,
    health_total_businesses INTEGER,
    z_score_health FLOAT,
    polling_total INTEGER,
    z_score_pol FLOAT,
    primary_per_1000 FLOAT, 
    z_score_primary FLOAT,
    secondary_per_1000 FLOAT,
    z_score_secondary FLOAT,
    future_per_1000 FLOAT,
    z_score_future FLOAT,
    x_val FLOAT,
    earners INTEGER,
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER,
    sigmoid_val FLOAT,
    geom GEOMETRY(MULTIPOLYGON,4326)

);"""

query(conn, sql)

In [ ]:
unextended_data.to_sql('unextended_data', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4326)})
query(conn, "select * from unextended_data")

In [ ]:
sql = """
CREATE INDEX unextended_data_geom_idx ON unextended_data USING GIST(geom);
"""
query(conn, sql)

In [ ]:
extended_data = pd.read_sql_query("""

WITH filtered_stops AS (
	SELECT sa2_boundaries.sa2_code21 AS sa2_code, COUNT(*) AS stops_total FROM sa2_boundaries 
	JOIN stops
	ON ST_Contains(sa2_boundaries.geom, stops.geom)
	GROUP BY sa2_code21
),

filtered_businesses AS (
	SELECT a.sa2_code, 
		a.sa2_name, 
		a.total_businesses AS retail_total_businesses, 
		b.total_businesses as health_total_businesses, total_people, 
		(cast(b.total_businesses as float) / total_people) * 1000 as health_businesses_per_1000, 
		(cast(a.total_businesses as float) / total_people) * 1000 as retail_businesses_per_1000 
	FROM businesses a, businesses b, population
	WHERE a.sa2_code = b.sa2_code and b.sa2_code = population.sa2_code
	AND a.industry_name = 'Health Care and Social Assistance' AND b.industry_name = 'Retail Trade'
	and total_people >= 100
), 

filtered_pol AS (
	SELECT sa2_code21, count(*) AS polling_total FROM sa2_boundaries 
	JOIN polling
	ON ST_Contains(sa2_boundaries.geom, polling.geom)
	GROUP BY sa2_code21
), 
	
filtered_primary AS ( 
	select sa2_code21, 
		COUNT(catchments_primary.use_id) AS primary_catchments
	FROM sa2_boundaries 
	join catchments_primary
	on ST_intersects(sa2_boundaries.geom, catchments_primary.geom)
	group by sa2_code21
), 

primary_reduced_population as (
select sa2_code,
	(cast(primary_catchments as float)/("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") ) * 1000 as primary_per_1000
	from filtered_primary
	join population 
	on filtered_primary.sa2_code21 = population.sa2_code
	where total_people >= 100	
),

filtered_secondary AS (
	SELECT sa2_code21, COUNT(*) AS secondary_catchments FROM sa2_boundaries 
	JOIN catchments_secondary
	ON ST_Intersects(sa2_boundaries.geom, catchments_secondary.geom)
	GROUP BY sa2_code21
), 

secondary_reduced_population as (
select sa2_code,
	(cast(secondary_catchments as float)/("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") ) * 1000 as secondary_per_1000
	from filtered_secondary
	join population 
	on filtered_secondary.sa2_code21 = population.sa2_code
	where total_people >= 100	
),

filtered_future AS (
	SELECT sa2_code21, COUNT(*) AS future_catchments FROM sa2_boundaries 
	JOIN catchments_future
	ON ST_intersects(sa2_boundaries.geom, catchments_future.geom)
	GROUP BY sa2_code21
),

future_reduced_population as (
select sa2_code,
	(cast(future_catchments as float)/("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") ) * 1000 as future_per_1000
	from filtered_future
	join population 
	on filtered_future.sa2_code21 = population.sa2_code
	where total_people >= 100	
),

filtered_toilets AS (
	SELECT sa2_code21, COUNT(*) AS toilet_total FROM sa2_boundaries 
	JOIN toilets
	ON ST_contains(sa2_boundaries.geom, toilets.geom)
	GROUP BY sa2_code21
),

filtered_unemployment AS (
	SELECT population.sa2_code, 
	population.sa2_name, 
	"per_unempl", 
	(100 - per_unempl) as employment_percentage,
	(total_people - "0-4_people" - "5-9_people" - "10-14_people") AS adult_population, 
	ROUND(((total_people - "0-4_people" - "5-9_people" - "10-14_people") - ((per_unempl / 100) * (total_people - "0-4_people" - "5-9_people" - "10-14_people")))) AS total_employed 
	FROM  population
	join unemployment
	ON population.sa2_code = unemployment.sa2_code
	where total_people >= 100
),

filtered_traffic as (
	SELECT sa2_code21, sa2_name21, COUNT(*) AS traffic_camera_count FROM sa2_boundaries 
	JOIN traffic
	ON ST_intersects(sa2_boundaries.geom, traffic.geom)
	GROUP BY sa2_code21, sa2_name21
),


zscore_stops AS (
SELECT sa2_code, 
	stops_total,
	(stops_total - (select avg(stops_total) from filtered_stops)) / (select stddev(stops_total) from filtered_stops) as z_score_stops
FROM filtered_stops
), 

zscore_businesses AS (
SELECT sa2_code, 
	retail_total_businesses, 
	(retail_businesses_per_1000 - (select avg(retail_businesses_per_1000) from filtered_businesses)) / (select stddev(retail_businesses_per_1000) from filtered_businesses) as z_score_retail, 
	health_total_businesses,
	(health_businesses_per_1000 - (select avg(health_businesses_per_1000) from filtered_businesses)) / (select stddev(health_businesses_per_1000) from filtered_businesses) as z_score_health 
FROM filtered_businesses
),

zscore_pol AS (
SELECT sa2_code21, 
	polling_total,
	(polling_total - (select avg(polling_total) from filtered_pol)) / (select stddev(polling_total) from filtered_pol) as z_score_pol
FROM filtered_pol
), 

zscore_primary AS (
SELECT sa2_code, 
	primary_per_1000,
	(primary_per_1000 - (select avg(primary_per_1000) from primary_reduced_population)) / (select stddev(primary_per_1000) from primary_reduced_population) as z_score_primary
FROM primary_reduced_population
),

zscore_secondary AS (
SELECT sa2_code, 
	secondary_per_1000,
	(secondary_per_1000 - (select avg(secondary_per_1000) from secondary_reduced_population)) / (select stddev(secondary_per_1000) from secondary_reduced_population) as z_score_secondary
FROM secondary_reduced_population
),

zscore_future AS (
SELECT sa2_code, 
	future_per_1000,
	(future_per_1000 - (select avg(future_per_1000) from future_reduced_population)) / (select stddev(future_per_1000) from future_reduced_population) as z_score_future
FROM future_reduced_population
),

zscore_toilets AS (
SELECT sa2_code21, 
	toilet_total,
	(toilet_total - (select avg(toilet_total) from filtered_toilets)) / (select stddev(toilet_total) from filtered_toilets) as z_score_toilets
FROM filtered_toilets
), 

zscore_employment AS (
SELECT sa2_code, 
	employment_percentage,
	(employment_percentage - (select avg(employment_percentage) from filtered_unemployment)) / (select stddev(employment_percentage) from filtered_unemployment) as z_score_employment
FROM filtered_unemployment
),

zscore_traffic AS (
SELECT sa2_code21, 
	traffic_camera_count,
	(traffic_camera_count - (select avg(traffic_camera_count) from filtered_traffic)) / (select stddev(traffic_camera_count) from filtered_traffic) as z_score_traffic
FROM filtered_traffic
), 


total_zscore as ( 
	SELECT 
		sa2_boundaries.sa2_code21,
        sa2_boundaries.geom,
        sa2_boundaries.sa2_name21,
		stops_total, 
		z_score_stops,
		retail_total_businesses,
		z_score_retail,
		health_total_businesses, 
		z_score_health,
		polling_total,
		z_score_pol,
		primary_per_1000, 
		z_score_primary,
		secondary_per_1000, 
		z_score_secondary,
		future_per_1000, 
		z_score_future,
        toilet_total,
        z_score_toilets,
        employment_percentage, 
        z_score_employment,
        traffic_camera_count,
        z_score_traffic,
		(COALESCE(z_score_stops, 0) + COALESCE(z_score_retail, 0) + COALESCE(z_score_health, 0) + COALESCE(z_score_pol, 0) + COALESCE(z_score_primary, 0) + COALESCE(z_score_secondary, 0) + COALESCE(z_score_future, 0) + COALESCE(z_score_toilets, 0) + COALESCE(z_score_employment, 0) + COALESCE(z_score_traffic, 0)) AS x_val
	FROM sa2_boundaries
	LEFT JOIN zscore_stops
	ON sa2_boundaries.sa2_code21 = zscore_stops.sa2_code
	LEFT JOIN zscore_businesses 
	ON sa2_boundaries.sa2_code21 = zscore_businesses.sa2_code
	LEFT JOIN zscore_pol 
	ON sa2_boundaries.sa2_code21 = zscore_pol.sa2_code21
	LEFT JOIN zscore_primary 
	ON sa2_boundaries.sa2_code21 = zscore_primary.sa2_code
	LEFT JOIN zscore_secondary
	ON sa2_boundaries.sa2_code21 = zscore_secondary.sa2_code
	LEFT JOIN zscore_future
	ON sa2_boundaries.sa2_code21 = zscore_future.sa2_code
	LEFT JOIN zscore_toilets
	ON sa2_boundaries.sa2_code21 = zscore_toilets.sa2_code21
	LEFT JOIN zscore_employment
	ON sa2_boundaries.sa2_code21 = zscore_employment.sa2_code
    LEFT JOIN zscore_traffic
	ON sa2_boundaries.sa2_code21 = zscore_traffic.sa2_code21
    LEFT JOIN income
    ON sa2_boundaries.sa2_code21 = income.sa2_code
)

select *, (1 / (1 + exp(-"x_val"))) as sigmoid_val from total_zscore




""",conn)

In [ ]:
sql = """

DROP TABLE IF EXISTS extended_data;
CREATE TABLE extended_data (
    sa2_code21 INTEGER PRIMARY KEY,
    sa2_name21 VARCHAR(100),
    stops_total INTEGER,
    z_score_stops FLOAT,
    retail_total_businesses INTEGER,
    z_score_retail FLOAT,
    health_total_businesses INTEGER,
    z_score_health FLOAT,
    polling_total INTEGER,
    z_score_pol FLOAT,
    primary_per_1000 FLOAT, 
    z_score_primary FLOAT,
    secondary_per_1000 FLOAT,
    z_score_secondary FLOAT,
    future_per_1000 FLOAT,
    z_score_future FLOAT,
    toilet_total INTEGER,
    z_score_toilets FLOAT,
    employment_percentage FLOAT, 
    z_score_employment FLOAT,
    traffic_camera_count INTEGER,
    z_score_traffic FLOAT,
    x_val FLOAT,
    earners INTEGER,
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER,
    sigmoid_val FLOAT,
    geom GEOMETRY(MULTIPOLYGON,4326)

);"""

query(conn, sql)

In [ ]:
extended_data.to_sql('extended_data', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4326)})
query(conn, "select * from extended_data")

In [ ]:
sql = """
CREATE INDEX extended_data_geom_idx ON extended_data USING GIST(geom);
"""
query(conn, sql)

In [ ]:
sa2map_unextended = gpd.read_postgis("SELECT geom, sigmoid_val FROM unextended_data ", conn, geom_col='geom')
sa2map_unextended.plot(column='sigmoid_val', legend=True, figsize=(10,10))
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Unextended Data "Well Resourced" Score Heatmap')
plt.savefig('unextended heatmap.png', dpi = 300)
plt.show()

In [ ]:
sa2map_extended = gpd.read_postgis("SELECT geom, sigmoid_val FROM extended_data ", conn, geom_col='geom')
sa2map_extended.plot(column='sigmoid_val', legend=True, figsize=(10,10))
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Extended Data "Well Resourced" Score Heatmap')
plt.savefig('extended heatmap.png', dpi = 300)
plt.show()

### Correlation Analysis

In [ ]:
#filtering out NaN income rows to include only income data that matches with sa2 boundaries
unextended_correlation_data = unextended_data[unextended_data['mean_income'].notna()]
extended_correlation_data = extended_data[extended_data['mean_income'].notna()]

In [ ]:
#Calculating correlation coefficient for original unextended data
r_coefficient = pearsonr(unextended_correlation_data['median_income'], unextended_correlation_data['sigmoid_val'])
print('r coefficient:',r_coefficient[0])

#Creating scatter plot of median income and wellness score and line of best fit for unextended data, with correlation coefficient 
plt.scatter(unextended_data['median_income'], unextended_data['sigmoid_val'])
a, b = np.polyfit(unextended_correlation_data['median_income'], unextended_correlation_data['sigmoid_val'], 1)
lobf = a*unextended_correlation_data['median_income']+b
plt.plot(unextended_correlation_data['median_income'],lobf, color='red', linestyle = 'dashed')
plt.xlabel('Median Income')
plt.ylabel('"Well Resourced" Score')
plt.title('SA2 Unextended Data Score vs Median Income')
plt.text(16500, 0.9, f'Correlation Coefficient: {r_coefficient[0]:.2f}', fontsize=10)
plt.savefig('unextended.png', dpi = 300)
plt.show()


In [ ]:
#Calculation of r coefficient for extended data
r_coefficient_ext = pearsonr(extended_correlation_data['median_income'], extended_correlation_data['sigmoid_val'])
print('r_coefficient:',r_coefficient_ext[0])

#Scatter plot of median income and wellness score for extended data
plt.scatter(extended_correlation_data['median_income'], extended_correlation_data['sigmoid_val'])
a, b = np.polyfit(extended_correlation_data['median_income'], extended_correlation_data['sigmoid_val'], 1)
lobf = a*extended_correlation_data['median_income']+b
plt.plot(extended_correlation_data['median_income'],lobf, color='red', linestyle = 'dashed')
plt.xlabel('Median Income')
plt.ylabel('"Well Resourced" Score')
plt.title('SA2 Extended Data Score vs Median Income')
plt.text(16500, 0.9, f'Correlation Coefficient: {r_coefficient_ext[0]:.2f}', fontsize=10)
plt.savefig('extended.png', dpi = 300)
plt.show()

